<a href="https://www.kaggle.com/code/juzykaggle/nn-sequential-optimization?scriptVersionId=139406102" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
# filling missing values whith median
for i in train.columns:
    if train[i].isna().sum()>0:
        train[i] = train[i].fillna(train[i].median())

In [ ]:
# get dummies for categorial feature
train['EJ'] = pd.get_dummies(train['EJ'], drop_first=True)

In [ ]:
train = train.drop(['Id'], axis=1)

In [ ]:
train

In [ ]:
X = train.drop(['Class'], axis=1)
y = train.Class

# Compile base model by Sequential

In [ ]:
# baseline model with 56 neurons (such as number of features)
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(56, input_dim=56, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# evaluate model 
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
print(results.mean()*100, results.std()*100)

Neural network models are specifically designed to provide consistent input measurements at scale as well as at distribution.
An effective scheme for preparing data for tabular data when building neural network models is standardization. 
Here, the data is scaled such that the mean for each attribute is 0 and the standard deviation is 1.
We use the scikit-learn standard to process our dataset using the StandardScaler for the class.
Instead of performing standardization on the entire dataset, 
it is good practice to train the training data standardization procedure during a cross-validation pass 
and use the trained standardization to prepare an "invisible" test set. 
This makes standardization a model preparation step in the cross-validation process 
and prevents the algorithm from using knowledge of "invisible" data during evaluation, 
knowledge that can be transferred from the data preparation scheme, such as a sharper distribution.

In [ ]:
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean()*100, results.std()*100)

We see a small but very pleasant rise in medium accuracy.
In next experiment, we take our base model with 56 neurons in the hidden layer and halve it to 28. 
This will put pressure on the network during training to choose the most important input structure for the model.

In [ ]:
# smaller model
def create_smaller_model():
    model = Sequential()
    model.add(Dense(28, input_dim=56, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Standartized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

The model worked a little more closely
Let's check bigger model whith adding another layer

In [ ]:
# bigger model
def create_bigger_model():
    model = Sequential()
    model.add(Dense(56, input_dim=56, kernel_initializer='normal', activation='relu'))
    # adding layer
    model.add(Dense(26, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standartize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_bigger_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Standartized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Increasing the layers in the network leads to a decrease in the score

Tuning the hyperparameters

In [ ]:
# tuning of batch_size and epochs
param_batch_epochs = {'batch_size': range(5, 20, 5), 'epochs': range(50, 200, 50)}
model_batch_epochs = KerasClassifier(build_fn=create_smaller_model, verbose=0)
grid_batch_epochs = GridSearchCV(model_batch_epochs, param_grid=param_batch_epochs)
batch_epochs_res = grid_batch_epochs.fit(X, y)

In [ ]:
batch_epochs_res.best_score_

In [ ]:
batch_epochs_res.best_params_

In [ ]:
# tuning weights initialization
def create_smaller_model_for_tune(init):
    model = Sequential()
    model.add(Dense(28, input_dim=56, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

init = ['uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_weights = dict(init=init)
model_weights = KerasClassifier(build_fn=create_smaller_model_for_tune, batch_size = 15,epochs = 150, verbose=0)
grid_weights = GridSearchCV(model_weights, param_grid=param_weights)
weights_res = grid_weights.fit(X, y)

In [ ]:
weights_res.best_score_

In [ ]:
weights_res.best_params_

In [ ]:
# tuning Dropout Regularization
def create_smaller_model_for_tune_dropout(dropout_rate, weight_constraint):
    model = Sequential()
    model.add(Dense(28, input_dim=56, kernel_initializer='uniform', activation='relu', kernel_constraint=tf.keras.constraints.max_norm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
param_dropout = dict(dropout_rate = dropout_rate, weight_constraint=weight_constraint)

model_dropout = KerasClassifier(build_fn=create_smaller_model_for_tune_dropout, verbose=0)
grid_dropout = GridSearchCV(model_dropout, param_grid=param_dropout)
dropout_res = grid_dropout.fit(X, y)

In [ ]:
dropout_res.best_params_

In [ ]:
dropout_res.best_score_

In [ ]:
# final model
def create_final_model():
    model = Sequential()
    model.add(Dense(28, input_dim=56, kernel_initializer='uniform', activation='relu'), tf.keras.constraints.max_norm(4))
    model.add(Dropout(0.6))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_final_model, epochs=150, batch_size=15, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))